In [1]:
import pandas as pd
import MySQLdb

In [2]:
#hide_me
db = MySQLdb.connect(host="172.18.55.99",    # tu host, usualmente localhost
                     user="comandato",         # tu usuario
                     passwd="comandato123",  # tu password
                     db="call_center")        # el nombre de la base de datos
cur=db.cursor()

In [3]:
consulta="""SELECT id FROM call_center.campaign where datetime_init between '2019-02-01' and '2019-03-11'"""

In [4]:
id_camp = pd.read_sql(consulta,con=db)

In [5]:
contadoreliminados=0
for row in id_camp.itertuples():
    del_query1="""DELETE FROM campaign_form 
WHERE
    id_campaign = '{0}';""".format(row.id)
    del_query2="""DELETE FROM call_recording 
WHERE
    id_call_outgoing IN (SELECT 
        id
    FROM
        calls
    
    WHERE
        id_campaign = '{0}');""".format(row.id)
    del_query3="""DELETE FROM call_attribute 
WHERE
    id_call IN (SELECT 
        id
    FROM
        calls
    
    WHERE
        id_campaign = '{0}');""".format(row.id)
    del_query4="""DELETE FROM form_data_recolected 
WHERE
    id_calls IN (SELECT 
        id
    FROM
        calls
    
    WHERE
        id_campaign = '{0}');""".format(row.id)
    del_query5="""DELETE call_progress_log FROM call_progress_log,
    calls 
WHERE
    call_progress_log.id_call_outgoing = calls.id
    AND calls.id_campaign = '{0}';""".format(row.id)
    del_query6="""DELETE FROM calls 
WHERE
    id_campaign ='{0}';""".format(row.id)
    del_query7="""
DELETE FROM campaign 
WHERE
    id = '{0}'""".format(row.id)
    try:
        cur.execute(del_query1)
        db.commit()
        cur.execute(del_query2)
        db.commit()
        cur.execute(del_query3)
        db.commit()
        cur.execute(del_query4)
        db.commit()
        cur.execute(del_query5)
        db.commit()
        cur.execute(del_query6)
        db.commit()
        cur.execute(del_query7)
        db.commit()
        contadoreliminados=contadoreliminados+1
    except:
        db.rollback()
print(str(contadoreliminados))

16
